Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-12 16:50:04--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-12 16:50:04--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  87.2KB/s    in 0.7s    

2021-10-12 16:50:06 (87.2 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/u/0/uc?id=1mTpNls48M3NcsJzwDAPGyvZLMileQZdj&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1mTpNls48M3NcsJzwDAPGyvZLMileQZdj
To: /content/2020CSY7657_A_model.zip
100% 23.2M/23.2M [00:01<00:00, 19.9MB/s]


In [3]:
!unzip 2020CSY7657_A_model.zip

Archive:  2020CSY7657_A_model.zip
  inflating: 2020CSY7657_A_model/field.pkl  
   creating: 2020CSY7657_A_model/results/
   creating: 2020CSY7657_A_model/results/bilstm/
   creating: 2020CSY7657_A_model/results/bilstm/wsd/
   creating: 2020CSY7657_A_model/results/bilstm/wsd/.ipynb_checkpoints/
  inflating: 2020CSY7657_A_model/results/bilstm/wsd/best-bilstm-wsd-params.pt  


In [4]:
import sys
import re
def cleaning(text):
  text= text.lower()
  text=text.strip()
  # text=text.replace('[^\w\s]','')
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  replace_dot = re.compile("\.")
  text = replace_dot.sub(' ', text)
  replace_q = re.compile("\?")
  text = replace_q.sub(' ', text)
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
  text = REPLACE_BY_SPACE_RE.sub('' , text)
  text = BAD_SYMBOLS_RE.sub('',text) 
  text= text.strip() 
  return text

In [5]:
import pandas as pd
df_val = pd.read_csv('test.data.txt', sep ="\t",names = ['word', 'status', 'position','sentence1','sentence2'])
df_val['sentence1']=df_val['sentence1'].apply(cleaning)
df_val['sentence2']=df_val['sentence2'].apply(cleaning)
df_test = pd.DataFrame()
df_test['data'] = df_val['sentence1'] +" "+ df_val['sentence2']+" "+ df_val['word']+" "+ df_val['word']


In [6]:
import torch   
from torchtext.legacy import data
import spacy
import dill
import torch.nn as nn

In [7]:
with open('./2020CSY7657_A_model/field.pkl', 'rb') as in_strm:
    TEXT = dill.load(in_strm)
fields = [('text',TEXT)]

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class bilstm(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        super().__init__()          
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        dense_outputs=self.fc(hidden)
        outputs=self.act(dense_outputs)        
        return outputs

In [10]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 300
num_hidden_nodes = 128
num_output_nodes = 1
num_layers = 3
bidirection = True
dropout = 0.2

model = bilstm(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [11]:
path='./2020CSY7657_A_model/results/bilstm/wsd/best-bilstm-wsd-params.pt'
checkpoint = torch.load(path)
options = checkpoint['options']
accuracy = checkpoint['accuracy']
model.load_state_dict(checkpoint['model_dict'])

<All keys matched successfully>

In [12]:
accuracy

tensor(0.6032)

In [13]:
model.eval()
nlp = spacy.load('en')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          
    length = [len(indexed)]                                    
    tensor = torch.LongTensor(indexed).to(device)              
    tensor = tensor.unsqueeze(1).T                             
    length_tensor = torch.LongTensor(length)                   
    prediction = model(tensor, length_tensor)                  
    return prediction.item()                                   

In [14]:
outputfile = "output.txt"
ans = ""
file1 = open(outputfile,"w+")
for i in range(len(df_test)):
  output = predict(model,df_test.iloc[i].data)
  if round(output) == 1:
    ans = "T"
  else:
    ans = "F"
  file1.write(ans)
  file1.write("\n")  
file1.close()

In [15]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-12 16:50:51--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-12 16:50:52--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-12 16:50:53 (48.2 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.5642633228840125
